Project 1 - Terrence Cummings

This notebook is used to make gmaps heatmaps of housing prices. Separate notebook used in order to isolate google API calls from other work so as not run up bill troubleshooting analytics etc.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import gmaps.datasets
import os
import gmaps

# Import API key
from api_keys import g_key

#READ SUMMARY ASSESSED HOME VALUE STATS CREATED IN ANALYTICS NOTEBOOK
msp_community_summary_df = pd.DataFrame(pd.read_csv("msp_community_summary.csv"))

#READ IN FILE THAT MAPS NEIGHBORHOOD TO COMMUNITY AND LAT/LONG
community_nbhd_df = pd.DataFrame(pd.read_csv('../community_nbhd.csv'))

#READ IN DATA THAT SHOWS ASSESSED VALUE PER SQFT FOR EACH NEIGHBORHOOD
nbhd_val_tot_sqft_df = pd.DataFrame(pd.read_csv('nbhd_val_tot_sqft.csv'))

#CREATE DF THAT INCLUDES NEIGHBORHOOD, COMMUNITY, ASSESSED VALUE PER SQFT, AND LAT LONG
nbhd_val_latlng = community_nbhd_df.merge(nbhd_val_tot_sqft_df, left_on='NEIGHBORHOOD', right_on='neighborhood')

#CLEANUP AFTER MERGE
del nbhd_val_latlng['neighborhood']
del nbhd_val_latlng['Unnamed: 0']

nbhd_val_latlng

,NEIGHBORHOOD,COMMUNITY,Google Lat,Google Long,Number of Homes,val_tot_sqft
0,BRYN MAWR,CALHOUN-ISLE,44.973721,-93.308377,1114,178.574770
1,CARAG,CALHOUN-ISLE,44.941056,-93.291062,831,192.321806
2,CEDAR-ISLES-DEAN,CALHOUN-ISLE,44.954166,-93.321534,930,248.630467
3,EAST ISLES,CALHOUN-ISLE,44.955947,-93.300271,717,239.512465
4,ECCO,CALHOUN-ISLE,44.855850,-93.242525,567,233.393468
...,...,...,...,...,...,...
79,COMO,UNIVERSITY,44.987837,-93.222807,948,119.915567
80,MARCY-HOLMES,UNIVERSITY,44.985372,-93.244723,935,217.959893
81,MID CITY INDUSTRIAL AR,UNIVERSITY,44.998862,-93.217771,165,248.821044
82,NICOLLET IS/EAST BANK,UNIVERSITY,44.973677,-93.231077,738,312.173345


In [20]:
#CREATE A HEATMAP OF NEIGHBORHOOD ASSESSED HOME VALUES PER SQFT

#Configure gmaps and read api key
gmaps.configure(api_key=g_key)

#Grab the needed columns for the humidity-weighted heatmap
home_price_heatmap = nbhd_val_latlng[['Google Lat', 'Google Long', 'val_tot_sqft']]

#Center on Minneapolis!
minneapolis = [44.971404, -93.266287]

#Set locations based on city lat long from WeatherPy
locations = home_price_heatmap[['Google Lat', 'Google Long']]

#Weight the heatmap by humidity level
weights = home_price_heatmap['val_tot_sqft']

#Generate gmaps figure, add heatmap layer
fig = gmaps.figure(map_type='ROADMAP', center = minneapolis, zoom_level=13)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, point_radius=50, max_intensity=300))
fig


Figure(layout=FigureLayout(height='420px'))

In [ ]:
#NOT USED!! RETRIEVS GOOGLE ADDRESS AND LAT LNG FOR 2015-2019 HOME SALES

params = {
    "key": g_key,
}

for index, row in short_msp_home_sales_df.iterrows():
    # get address from df
    address = row["formatted_address"]
    address_new = address.replace("#", "%23")

    # change location each iteration while leaving original params in place
    params["address"] = f"{address_new}"
    
    #Base url and request
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        short_msp_home_sales_df.loc[index, 'Google Address'] = response['results'][0]['formatted_address']
        short_msp_home_sales_df.loc[index, 'Google Lat'] = response['results'][0]['geometry']['location']['lat']
        short_msp_home_sales_df.loc[index, 'Google Long'] = response['results'][0]['geometry']['location']['lng']
    except:
        print(f'Bad Google data for sale ID {index}')


In [ ]:
#NOT USED!! CREATES HEATMAP OF 2015-2019 HOME SALES VALUE

#Configure gmaps and read api key
gmaps.configure(api_key=g_key)

#Grab the needed columns for the humidity-weighted heatmap
home_sale_price_heatmap = short_msp_home_sales_df[['Google Lat', 'Google Long', 'gross_sale_price']]

#Center on Minneapolis!
minneapolis = [44.971404, -93.266287]

#Set locations based on city lat long from WeatherPy
locations = home_sale_price_heatmap[['Google Lat', 'Google Long']]

#Weight the heatmap by humidity level
weights = home_sale_price_heatmap['gross_sale_price']

#Generate gmaps figure, add heatmap layer
fig = gmaps.figure(map_type='HYBRID', center = minneapolis, zoom_level=9.5)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


In [6]:
#RETRIEVE GOOGLE MAPS LAT/LNG VALUES FOR EACH OF THE 80+ MSP NEIGHBORHOODS
params = {
    "key": g_key,
}

for index, row in community_nbhd_df.iterrows():
    # get address from df
    address = row["NEIGHBORHOOD"]+' MINNEAPOLIS'
    address_new = address.replace("#", "%23")

    # change location each iteration while leaving original params in place
    params["address"] = f"{address_new}"
    
    #Base url and request
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        community_nbhd_df.loc[index, 'Google Lat'] = response['results'][0]['geometry']['location']['lat']
        community_nbhd_df.loc[index, 'Google Long'] = response['results'][0]['geometry']['location']['lng']
    except:
        print(f'Bad Google data for sale ID {index}')

community_nbhd_df.to_csv('../community_nbhd.csv')